### Oracle Vector DB wrapped as a llama-index custom Vector Store

* ispired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* adding **reranker** after retrieval from Vector Store
* Demo on **Medicine Book**
* Demo on Python book

In [1]:
import logging
import sys

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document
from llama_index.postprocessor.cohere_rerank import CohereRerank

import oci
import ads

# only
import oracledb
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

from config import EMBED_MODEL
from config_private import COMPARTMENT_OCID, ENDPOINT, COHERE_API_KEY

In [2]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649


In [3]:
# for debugging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual
# EMBED_MODEL  from config

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=ads.auth.api_keys(oci_config),
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

# adding Cohere reranker
cohere_rerank = CohereRerank(api_key=COHERE_API_KEY, top_n=4)

In [5]:
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

In [6]:
v_store = OracleVectorStore(verbose=True)

In [7]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

In [8]:
index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

In [9]:
# added reranker to the chain
query_engine = index.as_query_engine(
    similarity_top_k=6,
    # after the query on the Vector Store we do reranking
    node_postprocessors=[cohere_rerank],
)

#### Using the wrapper for the DB Vector Store

In [10]:
question = "Which level of glucose in the blood are early indication of diabetes?"

In [11]:
# embed the query using OCI GenAI
query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=6)

#### Use our Vector Store DB

In [12]:
%%time

q_result = v_store.query(query_obj)

2023-12-23 22:36:23,316 - INFO - ---> Calling query on DB
2023-12-23 22:36:23,561 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d 
                            from VECTORS V, CHUNKS C
                            where C.ID = V.ID
                            order by d
                            FETCH FIRST 6 ROWS ONLY
2023-12-23 22:36:23,885 - INFO - Query duration: 0.6 sec.


CPU times: user 29.2 ms, sys: 10.4 ms, total: 39.6 ms
Wall time: 572 ms


In [ ]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):
    print(f"Dod. id: {id}")
    print(f"Similarity: {-sim}")
    print(n.text)
    print("")

#### Integrate in the bigger RAG picture

In [13]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print(response.response)
print("")

2023-12-23 22:43:47,462 - INFO - ---> Calling query on DB
2023-12-23 22:43:47,640 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d 
                            from VECTORS V, CHUNKS C
                            where C.ID = V.ID
                            order by d
                            FETCH FIRST 6 ROWS ONLY
2023-12-23 22:43:47,954 - INFO - Query duration: 0.5 sec.


Question: Which level of glucose in the blood are early indication of diabetes?
Polyuria, polydipsia, and blurred vision are early signs of diabetes, but blurry vision is more indicative of Type 2 diabetes, while polyuria and polydipsia are more characteristic of Type 1 diabetes. Fasting plasma glucose >= 126 mg/dL and random plasma glucose >= 200 mg/dL with symptoms are indicators of diabetes, but these numbers might be reached in severe cases only. Therefore, one should be watchful of any early signs and get checked by a medical professional if symptoms appear. 

Would you like me to go into more detail about the symptoms and treatment of diabetes? 

CPU times: user 130 ms, sys: 14.7 ms, total: 144 ms
Wall time: 6.55 s


#### pages with metadata (page_num)

In [ ]:
for node in response.source_nodes:
    print(f"{node.text}\npag:{node.metadata['page_label']}\n")